# Limpieza

## Parámetros

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
FILE_NAME = 'ile'

## Importar

### Entorno

In [2]:
from pandas import qcut
from IPython.display import display

### Clase

In [3]:
from mariachis.interrupcion_embarazo import InterrupcionEmbarazo

ile = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)
print(ile)

Directorio: 	/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


### Datos

In [4]:
df = ile.full_import(api=False)
df.sample()

Archivo con nombre ile.csv fue encontrado en /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data
Con 79403 renglones y 47 columnas


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
72760,2018.0,MARZO,2018-03-02,NaN,SOLTERA,25.0,IMSS,LICENCIATURA,TRABAJADORA DEL HOGAR NO REMUNERADA,OTRA,...,NaN,NaN,CONDON,NaN,COMPLETA,NaN,DFSSA003932,DFSSA003932,MEDICAMENTO,MISOPROSTOL + MIFEPRISTONA


## Diccionario de variables

### Limpieza

In [5]:
# Diccionario para aplicar limpieza por tipo de variable
clean_dict = {}

# Variables que o son 0 o >=1, es decir, no es relevante saber si son 2, 3, o más porque estarán agrupadas
clean_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']

# Numéricas que serán partidas en rangos para que los registros nulos no sean omitidos ni imputados
clean_dict['vars_num'] = {
    'edad': [9,11,15,18,21,25,29,35], # El algoritmo hará rangos: <= 9, 10-11, 12-15, ... , 30-35, >= 36
    'menarca': [7,11,12,13,14,15],
    'fsexual': [7,11,15,18,21],
    'sememb': [3,6,9,12,20],
    'nhijos': [0,1,2],
    'gesta': [0,1,2,3,4],
    'c_num': [0,1,2],
    'p_semgest': [0,5,6,7,8,9],
    'p_diasgesta': [0,1,2,3,4,5]
}

# Obtener sólo la primer palabra
clean_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Variables binarias: sólo tienen sí o no
clean_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile']

# Limpiar y mantener para clustering posterior
clean_dict['vars_clean_keep'] = ['motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep', 'procile_simplificada', 'procile_completa']

# Sólo limpiar acentos y caracteres especiales
clean_dict['vars_just_clean'] = ['entidad','alc_o_municipio']

# Agrupar clases de variables categóricas para que el clustering converga mejor y más rápido
clean_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',   # Se decide agrupar el término "DIVORCIADA" en "SEPARADX"
        'SEPARADA':'SEPARADX',     # Se decide agrupar el término "SEPARADA" en "SEPARADX" 
        'VIUDA':'SEPARADX',        # Se decide agrupar el término "VIUDA" en "SEPARADX" 
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'DESEMPLEADX',
    },
    'desc_derechohab':{
        'NINGUNO':'NINGUNO',
        'IMSS':'ALGUNO',
        'SEGURO POPULAR':'ALGUNO',
        'OTRA':'ALGUNO',
        'ISSSTE':'ALGUNO',
    },
    'nivel_edu':{
        'LICENCIATURA':'LICENCIATURA',
        'MAESTRIA':'POSGRADO',
        'DOCTORADO':'POSGRADO',
        'PREPARATORIA':'PREPARATORIA',
        'SECUNDARIA':'SECUNDARIA',
        'PRIMARIA':'PRIMARIA',
        'SIN ACCESO A LA EDUCACION FORMAL':'SIN_ACCESO',
    },
    'parentesco':{
        'PAREJA':'PAREJA',
        'EX-PAREJA':'PAREJA',
        'FAMILIAR MUJER':'FAMILIAR',
        'FAMILIAR HOMBRE':'FAMILIAR',
        'FAMILIAR SIN ESPECIFICAR':'FAMILIAR',
        'TUTOR O RESPONSABLE LEGAL':'FAMILIAR',
        'PERSONA CERCANA':'AMIGX',
        'PERSONA CERCANA MUJER':'AMIGX',
        'PERSONA CERCANA HOMBRE':'AMIGX',
        'AMIGA':'AMIGX',
        'AMIGO':'AMIGX',
        'CONOCIDA':'OTRX',
        'CONOCIDO':'OTRX',
        'RELACION LABORAL':'OTRX',
        'OTRA':'OTRX',
    },
    'religion':{
        'NINGUNA':'NINGUNA',
        'CATOLICA':'SI',
        'CRISTIANA':'SI',
        'OTRA':'SI',
        'MORMONA':'SI',
        'TESTIGA DE JEHOVA':'SI',
        'BUDISTA':'SI',
        'JUDIA':'SI',
        'ANGLICANA':'SI',
        'MUSULMANA':'SI',
    },
}

#### Auxiliar para crear rangos de variables numéricas

In [6]:
N_BINS = 6

aux_dict = {}
for col in clean_dict['vars_num']:
    # Utiliza el método de la Clase para limpiar la columna numérica
    new_col = df[col].map(ile.clean_number).astype(float)
    # Crea los rangos tal que cada bin contenga (aprox) el mismo número de registros
    orig_bins = qcut(new_col, q=N_BINS, retbins=True, duplicates='drop')[-1]
    # Guarda los puntos de corte (con excepción del último porque el algoritmo ya se encarga de la cota superior)
    aux_dict[col] = list(orig_bins[:-1])

# Así es la propuesta preliminar de rangos para las variables numéricas
display(aux_dict)

{'edad': [11.0, 20.0, 22.0, 25.0, 28.0, 32.0],
 'menarca': [0.0, 11.0, 12.0, 13.0, 14.0],
 'fsexual': [0.0, 15.0, 16.0, 17.0, 18.0, 19.0],
 'sememb': [0.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'nhijos': [0.0, 1.0, 2.0],
 'gesta': [0.0, 1.0, 2.0, 3.0, 4.0],
 'c_num': [0.0, 1.0, 2.0],
 'p_semgest': [0.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'p_diasgesta': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]}

### Creación de variables

In [7]:
# Diccionario para crear variables
vars_dict = {}

# Para calcular la diferencia entre edades
vars_dict['edad_1a_menst'] = 'menarca'
vars_dict['edad_vida_sex'] = 'fsexual'
vars_dict['edad_actual'] = 'edad'

# Para comparar los métodos anticonceptivos
vars_dict['metodo_antes'] = 'anticonceptivo'
vars_dict['metodo_despues'] = 'panticoncep'

## Transformar

### Limpieza y creación de variables

In [8]:
df, cluster_cols = ile.wrangling_ile(df, clean_dict, vars_dict, export_result=True)

Archivo: ile_limpio.csv fue exportado exitosamente en: /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


### Columnas para clustering

In [9]:
print(f'Las {len(cluster_cols)} columnas para clustering son:\n{cluster_cols}')
df.sample()

Las 38 columnas para clustering son:
['antes_vs_despues', 'antes_vs_despues_detalle', 'anticonceptivo', 'autoref', 'c_dolor', 'cconsejo', 'consejeria', 'desc_derechohab', 'desc_servicio', 'edad_vs_fsexual', 'edad_vs_menarca', 'edocivil_descripcion', 'fsexual_vs_menarca', 'motiles', 'naborto', 'ncesarea', 'nile', 'nivel_edu', 'npartos', 'ocupacion', 'p_consent', 'panticoncep', 'parentesco', 'procile_completa', 'procile_simplificada', 'rango_c_num', 'rango_edad', 'rango_fsexual', 'rango_gesta', 'rango_menarca', 'rango_nhijos', 'rango_p_diasgesta', 'rango_p_semgest', 'rango_sememb', 'religion', 'resultado_ile', 's_complica', 'tanalgesico']


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,rango_nhijos,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,rango_fsexual_vs_menarca,rango_edad_vs_menarca,rango_edad_vs_fsexual,antes_vs_despues,antes_vs_despues_detalle
5084,2016.0,SEPTIEMBRE,2016-09-13,DESCONOCIDO,SOLTERX,27.0,NINGUNO,SECUNDARIA,EMPLEADX,SI,...,02,>= 05,02,08,04,01 a 03,13 a 17,09 a 12,DIFERENTE,"antes: NINGUNO, después: DIU"
